In [1]:
import csv
import requests
from bs4 import BeautifulSoup

In [14]:
DOMAIN = "http://www.godaycare.com"
ENCODING = "iso-8859-1"
OUTPUT_FILENAME = 'data/childcare/odec.csv'
USER_AGENT_HEADER = {
    'User-Agent': 'KT Hobbs, Statistics Canada',
    'From': 'kaitlyn.hobbs@canada.ca'
}

In [3]:
CSV_DATA = []
DAY_CARE_NAME = ""
CITY_NAME = ""
PROV_TERR_NAME = ""
PROV_TERR_URLS = [
    "http://www.godaycare.com/alberta",
    "http://www.godaycare.com/british%20columbia",
    "http://www.godaycare.com/manitoba",
    "http://www.godaycare.com/new%20brunswick",
    "http://www.godaycare.com/newfoundland%20and%20labrador",
    "http://www.godaycare.com/northwest%20territories",
    "http://www.godaycare.com/nova%20scotia",
    "http://www.godaycare.com/nunavut",
    "http://www.godaycare.com/ontario",
    "http://www.godaycare.com/prince%20edward%20island",
    "http://www.godaycare.com/quebec",
    "http://www.godaycare.com/saskatchewan",
    "http://www.godaycare.com/yukon%20territory"
]

In [5]:
def request_page_source(prov_terr_url):
    """
    Request the page source for the prov/terr url
    """
    page_src = ""
    page_response = requests.get(prov_terr_url, timeout=5, headers=USER_AGENT_HEADER)
    if page_response.status_code == 200:
        page_src = page_response.content

    return page_src


def get_city_urls(bs):
    """
    Scrape a list of city URLs from a prov/terr page
    """
    global PROV_TERR_NAME
    PROV_TERR_NAME = ""

    heading = bs.find("div", class_="Heading")
    if heading:
        heading_h2 = heading.find('h2')
        if heading_h2:
            PROV_TERR_NAME = heading_h2.string[:-49]

    city_lists = bs.find_all('div', class_='menuCity')

    if city_lists:
        city_urls = []
        for city_list in city_lists:
            list_items = city_list.find_all('li')
            if list_items:
                for list_item in list_items:
                    list_item_link = list_item.find('a')
                    city_url = list_item_link.attrs['href']
                    if city_url:
                        city_urls.append(city_url)

    return city_urls

In [6]:
def request_city_page_source(city_source_url):
    """
    Get page source for the city page
    """
    city_url_fixed = city_source_url[11:].replace(' ', '%20')
    full_city_url = DOMAIN + city_url_fixed + '/1'
    city_page_src = ""
    city_page_response = requests.get(full_city_url, timeout=5, headers=USER_AGENT_HEADER)

    if city_page_response.status_code == 200:
        city_page_src = city_page_response.content

    return city_page_src


def get_city_day_care_urls(bs):
    """
    Scrape data related to city day care source
    - Get all city day care URLs
    - Get city name
    """
    dc_urls = []
    global CITY_NAME
    CITY_NAME = ""

    heading = bs.find('div', class_='Heading')
    if heading:
        heading_h2 = heading.find('h2')
        if heading_h2:
            CITY_NAME = heading_h2.string[36:]

    dc_links = bs.find_all('a', class_='dc_link')

    if dc_links:
        for dc_link in dc_links:
            dc_url = DOMAIN + dc_link.attrs['href'].replace(' ', '%20')
            dc_urls.append(dc_url)

    return dc_urls


def request_day_care_page_source(day_care_page_url):
    """
    Get the day care source page
    """
    dc_page_src = ""
    dc_page_response = requests.get(day_care_page_url, timeout=5, headers=USER_AGENT_HEADER)

    if dc_page_response.status_code == 200:
        dc_page_src = dc_page_response.content

    return dc_page_src


def get_day_care_details(bs):
    """
    Scrap specific details for day care
    Scraped Details:
    - Day care name
    - Day care province/territory
    - Day care city name
    - Day care location
    - Day care age groups
    - Day care phone number
    """
    global DAY_CARE_NAME
    DAY_CARE_NAME = ""

    dc_details = []

    # Get store day care name
    heading = bs.find('div', class_='Heading')
    if heading:
        heading_h2 = heading.find('h2')
        if heading_h2:
            DAY_CARE_NAME = heading_h2.string[:-34]
            dc_details.append(DAY_CARE_NAME.strip())

    # Store prov/terr name
    dc_details.append(PROV_TERR_NAME.strip())

    # Store city name
    dc_details.append(CITY_NAME.strip())

    # Information Div
    info_div = bs.find('div', id='InfoID')

    if info_div:
        # Store Location
        address = ""
        location = info_div.find('div', class_='InfoData')
        for content in location.contents:
            if str(content) == '<br/>':
                address += ' '
            else:
                address += content

        dc_details.append(address.strip())

        # Store Age Groups
        groups = info_div.find('div', class_='InfoDataWrap')
        if groups and groups.text:
            dc_details.append(groups.text.strip())
        else:
            dc_details.append('')

        # Store Telephone
        phone = info_div.find('div', class_='emphasis')
        if phone and phone.text:
            dc_details.append(phone.text.strip())
        else:
            dc_details.append('')

    print(dc_details)
    return dc_details

ON, PEI, QC
---

In [8]:
src = request_page_source("http://www.godaycare.com/ontario")
prov_terr_bs = BeautifulSoup(src, 'lxml')
# Get list of city urls for the province or territory
prov_terr_city_urls = get_city_urls(prov_terr_bs)

# Loop through collected city urls
for city in prov_terr_city_urls:
    city_src = request_city_page_source(city)

    # Create Beautiful Soup Object for City Page
    city_bs = BeautifulSoup(city_src, 'lxml')

    # Get City Day Care URLS
    day_care_urls = get_city_day_care_urls(city_bs)

    # Loop through day care urls
    for day_care_url in day_care_urls:
        # Request Day Care Source
        dc_src = request_day_care_page_source(day_care_url)

        # Create Beautiful Soup Object for Day Care Page
        dc_bs = BeautifulSoup(dc_src, 'lxml')

        # Get Day Care Details
        details = get_day_care_details(dc_bs)

        if details:
            CSV_DATA.append(details)

["Adrianna's All Natural Lil Monkey Daycare", 'Ontario', 'Acton', '33 Dawkins Crescent Acton ON', 'Toddler', '519-853-2831']
["Colleen's Daycare", 'Ontario', 'Acton', 'Greenore Cres. Acton ON', 'Infant Toddler Kindergarten Preschool Schoolage', '905-749-2211']
['Country Home Daycare', 'Ontario', 'Acton', '13016 5th Line Acton ON', 'Infant Toddler Kindergarten Preschool Schoolage', '905-691-2879']
['DAIRY DAYCARE', 'Ontario', 'Acton', '27 DAIRY DRIVE Acton ON', 'Infant Toddler Kindergarten Preschool Schoolage', '(519) 853-5779']
["DD's Daycare", 'Ontario', 'Acton', 'Kingham Rd. Acton ON', 'Infant Toddler Kindergarten Preschool Schoolage', '']
['Healthy Little Monkey', 'Ontario', 'Acton', 'Dawkins Crescent Acton ON', 'Infant Toddler Kindergarten Preschool Schoolage', '905.875.7399']
["Kitty's Home Daycare", 'Ontario', 'Acton', '14367 Fifth line Acton ON', 'Infant Toddler Kindergarten Preschool Schoolage', '4168848823']
["Lynn's Daycare", 'Ontario', 'Acton', '133 Crescent Acton ON', 'Todd

['Ancaster Small Fry Pre-School', 'Ontario', 'Ancaster', '184 Fiddlers Green Road Ancaster ON', 'Preschool', '905-304-8903']
['Childventures Early Learning Academy', 'Ontario', 'Ancaster', '1281 Mohawk Rd Ancaster ON', 'Preschool Schoolage', '905-304-1415']
['Childventures Early Learning Academy - Ancaster', 'Ontario', 'Ancaster', '1281 Mohawk Road W Ancaster ON', 'Infant Toddler Kindergarten Preschool Schoolage', '905-304-1415']
['Family Home Daycare', 'Ontario', 'Ancaster', '161 raymond rd Ancaster ON', 'Infant Toddler Kindergarten Preschool Schoolage', '289 815 0373']
['Higher Learning Montessori Preschool Inc.', 'Ontario', 'Ancaster', '211 Stonehenge Dr Ancaster ON', 'Preschool', '905-304-4080']
['Holy Name Of Mary - Before And After School Program', 'Ontario', 'Ancaster', '161 Meadowlands Blvd. Ancaster ON', 'Kindergarten Schoolage', '905-520-3716']
['Jacks And Jills Co-Operative Preschool Of Ancaster', 'Ontario', 'Ancaster', '265 Wilson Street East Ancaster ON', 'Preschool', '905

['YMCA Of Cambridge - Ayr YMCA After School Program', 'Ontario', 'Ayr', '105 Hall Street Ayr ON', 'Schoolage', '519-632-7424']
['YMCA Of Cambridge - Cedar Creek School Age Program', 'Ontario', 'Ayr', '55 Hilltop Drive Ayr ON', 'Schoolage', '519-632-5255']
["South East Grey Community Outreach Inc. (Kids 'N Us - Ayton)", 'Ontario', 'Ayton', '574 Louisa St Ayton ON', 'Preschool Schoolage', '519-665-2022']
['Centre Pivot Du Triangle Magique De Rayside Balfour - Ã\x89cole Ste-Marie', 'Ontario', 'Azilda', '25 Rue Marier Azilda ON', 'Infant Toddler Preschool Schoolage', '705-983-4203']
['Jessica Private Daycare', 'Ontario', 'Azilda', 'St. Alphonse st Azilda ON', 'Infant Toddler Kindergarten Preschool', '705-587-2231']
["Sheena's House", 'Ontario', 'Azilda', 'Prevost St. South Azilda ON', 'Infant Toddler Kindergarten Preschool Schoolage', '705-587-2210']
["Catt's Childcare", 'Ontario', 'Baden', 'Jacob Cressman Dr Baden ON', 'Infant Toddler Kindergarten Preschool', '519-616-4731']
['Creative Be

['Sonshine Day Care-Latch Key - B-A School', 'Ontario', 'Belle River', '370 St. Peter Street Belle River ON', 'Kindergarten Schoolage', '519-728-0115']
['Your Wooden Treehouse - Day Care', 'Ontario', 'Belle River', '220 Church St. Belle River ON', 'Toddler Preschool Schoolage', '519-728-4441']
["Abigail's Learning Centre", 'Ontario', 'Belleville', '119 Station Street Belleville ON', 'Infant Toddler Preschool', '613-961-1884']
["Abigail's Learning Centre Early Start Pilot", 'Ontario', 'Belleville', '75 Ritchie Avenue Belleville ON', 'Kindergarten Schoolage', '613-961-1884']
['Belleville Day Nursery', 'Ontario', 'Belleville', '281 John Street Belleville ON', 'Toddler Preschool Schoolage', '613-962-6957']
["Belleville Day Nursery's Before And After Program At St. Michael's", 'Ontario', 'Belleville', '273 Church Street Belleville ON', 'Schoolage', '613-921-1617']
['Belleville Montessori School', 'Ontario', 'Belleville', '28 Oriole Park Avenue Belleville ON', 'Preschool', '613-966-7672']
['

['Macaulay Tree House Day Nursery', 'Ontario', 'Bracebridge', '50 The Granite Bluff Bracebridge ON', 'Infant Toddler Kindergarten Preschool', '705-645-1956']
['Macaulay Tree House Day Nursery (macaulay Public School)', 'Ontario', 'Bracebridge', '1270 Cedar Lane Bracebridge ON', 'Kindergarten Schoolage', '705-645-1956']
['Macaulay Tree House Day Nursery - Monck School Age Program', 'Ontario', 'Bracebridge', '250 Wellington Street Bracebridge ON', 'Kindergarten Schoolage', '705-645-1956']
['Montessori School Of Bracebridge', 'Ontario', 'Bracebridge', '265 Maple Street Bracebridge ON', 'Kindergarten Preschool Schoolage', '705-645-1443']
["Muskoka Family Focus And Children's Place-Bracebridge Children's Place", 'Ontario', 'Bracebridge', '2 Tamarack Trail Bracebridge ON', 'Toddler Kindergarten Preschool Schoolage', '705-645-3362']
["Muskoka Family Focus And Children's Place-Bracebridge Public School Childre", 'Ontario', 'Bracebridge', '90 McMurray Street Bracebridge ON', 'Kindergarten Presc

...

["Kelly's Place Home Daycare", 'Ontario', 'Stoney Creek', 'Bankfield Crescent Stoney Creek ON', 'Infant Toddler Preschool Schoolage', '905-930-7501']
["Little Monky's Home Daycare", 'Ontario', 'Stoney Creek', '563 Fifty Rd Stoney Creek ON', 'Infant Toddler Preschool', '905 930 9692']
['Michelle Gallagher', 'Ontario', 'Stoney Creek', 'Westhampton Way Stoney Creek ON', 'Toddler Preschool', '289-237-6979']
['Adonai Child Care', 'Ontario', 'Stoney Point', 'Meadow Wood Cres Stoney Point ON', 'Infant Toddler Preschool', '2894899411']
['Casa Mia Montessori Childcare', 'Ontario', 'Stouffville', 'Isabella Garden Lane Stouffville ON', 'Toddler Preschool', '416-419-5736']
['EDUKids Child Care - Christ Church', 'Ontario', 'Stouffville', '254 Sunset Blvd Stouffville ON', 'Infant Toddler Kindergarten Preschool Schoolage', '905-6401-1773']
['Home Sweet Home Day Care', 'Ontario', 'Stouffville', 'Yakefarm Blvd. Stouffville ON', 'Infant Toddler Kindergarten Preschool Schoolage', '905-591-1303']
['Jens D

['Little Steps Daycare', 'Ontario', 'Thunder Bay', '729 North James Street Thunder Bay ON', 'Infant Toddler Kindergarten Preschool Schoolage', '807 475 7124']
['Private Home Daycare', 'Ontario', 'Thunder Bay', '131 Churchill Dr W Thunder Bay ON', 'Toddler Kindergarten Preschool Schoolage', '625-8928']
["Sara's Home Daycare", 'Ontario', 'Thunder Bay', '111 Ravenwood Cr. Thunder Bay ON', 'Toddler Preschool', '684-0445']
["Mrs Beth's Play House", 'Ontario', 'Tilbury', 'Canal St.West Tilbury ON', 'Infant Toddler Preschool', '519-784-1327']
["Amanda's Child Care", 'Ontario', 'Tillsonburg', '29 Langrell Ave Tillsonburg ON', 'Toddler Kindergarten Preschool', '(519) 688-2768']
['Butter And Noodles Home Daycare', 'Ontario', 'Tillsonburg', 'Lisgar Ave Tillsonburg ON', 'Infant Toddler Kindergarten Preschool Schoolage', '519-688-3985']
["Little Tot's Childcare", 'Ontario', 'Tillsonburg', '45 Trillium Dr Tillsonburg ON', 'Infant Toddler Kindergarten Preschool Schoolage', '226-377-4452']
['South Rid

['Garderie Ã\x83Â\xa0 Vars Chez Chantal', 'Ontario', 'Vars', '1950 Rollin Place Vars ON', 'Toddler Kindergarten Preschool Schoolage', '613-429-0249']
['Garderie Du Soleil', 'Ontario', 'Vars', '5272 Frank Kenny Vars ON', 'Infant Toddler Preschool', '6138353727']
['Childventures Early Learning Academy - Vaughan', 'Ontario', 'Vaughan', '9306 Bathurst St. Vaughan ON', 'Infant Toddler Kindergarten Preschool Schoolage', '905-303-5090']
['Cozytime Daycare', 'Ontario', 'Vaughan', '9222 Keele Street Vaughan ON', 'Infant Toddler Kindergarten Preschool', '416 602 3811']
['Discovery Point Nursery And Academy', 'Ontario', 'Vaughan', '10501 Weston Road Vaughan ON', 'Infant Toddler Kindergarten Preschool', '905-727-3568']
['Fantasy Lan Childcare', 'Ontario', 'Vaughan', '12 Camino dr Vaughan ON', 'Infant Toddler Kindergarten Preschool Schoolage', '647 997 8061']
['Fantasy Land Childcare', 'Ontario', 'Vaughan', 'Camino dr Vaughan ON', 'Infant Toddler Kindergarten Preschool Schoolage', '647 997 8061']
[

['Inspiring Minds Early Learning Centre', 'Ontario', 'Wellesley', '1129 Henry St Wellesley ON', 'Toddler Kindergarten Preschool Schoolage', '519-656-1003']
["Leanne's Little Ones", 'Ontario', 'Wellesley', 'Deborah Glaister Line Wellesley ON', 'Infant Toddler Kindergarten Preschool Schoolage', '226-989-2600']
["Trish's Home Daycare", 'Ontario', 'Wellesley', 'Galena Street Wellesley ON', 'Infant Toddler Kindergarten Preschool Schoolage', '5195894878']
["Michelle Hughson's Child Care", 'Ontario', 'West Lorne', 'Walker Street West Lorne ON', 'Infant Toddler Kindergarten Preschool Schoolage', '']
['Happy Days Care', 'Ontario', 'Whitby', 'Watersdown Crescent Whitby ON', 'Infant Toddler Kindergarten Preschool Schoolage', '416-712-8191']
['A Loving Home Daycare - Durham', 'Ontario', 'Whitby', 'Puttingedge Drive Whitby ON', 'Toddler Kindergarten Preschool Schoolage', '9059255232']
['ABC 123 Home Daycare - Williamsburg Area', 'Ontario', 'Whitby', 'Abbyview Dr Whitby ON', 'Infant Toddler Kinderga

['Nature Daycare', 'Ontario', 'Zephyr', '13809 Concession Road 1 Zephyr ON', 'Infant Toddler Kindergarten Preschool Schoolage', '905-473-3946']


In [10]:
src = request_page_source("http://www.godaycare.com/prince%20edward%20island")
prov_terr_bs = BeautifulSoup(src, 'lxml')
# Get list of city urls for the province or territory
prov_terr_city_urls = get_city_urls(prov_terr_bs)

# Loop through collected city urls
for city in prov_terr_city_urls:
    city_src = request_city_page_source(city)

    # Create Beautiful Soup Object for City Page
    city_bs = BeautifulSoup(city_src, 'lxml')

    # Get City Day Care URLS
    day_care_urls = get_city_day_care_urls(city_bs)

    # Loop through day care urls
    for day_care_url in day_care_urls:
        # Request Day Care Source
        dc_src = request_day_care_page_source(day_care_url)

        # Create Beautiful Soup Object for Day Care Page
        dc_bs = BeautifulSoup(dc_src, 'lxml')

        # Get Day Care Details
        details = get_day_care_details(dc_bs)

        if details:
            CSV_DATA.append(details)

['Atlantic Child Care Centre', 'Prince Edward Island', 'Charlottetown', '18 Garfield Street Charlottetown PE', 'Infant Toddler Kindergarten Schoolage', '902-892-1400']
['Atlantic Child Care Centre', 'Prince Edward Island', 'Charlottetown', '119 Kent Street Charlottetown PE', 'Infant Toddler Kindergarten Schoolage', '902-894-4499']
['Basilica Recreation Centre', 'Prince Edward Island', 'Charlottetown', '200 Richmond Street Charlottetown PE', 'Schoolage', '902-566-9259']
['Brighton School Of Early Learning', 'Prince Edward Island', 'Charlottetown', '35 Fitzroy Street Charlottetown PE', 'Kindergarten', '902-892-2109']
['Campus Kids Child Care Centre', 'Prince Edward Island', 'Charlottetown', '550 University Avenue Charlottetown PE', 'Infant Toddler Kindergarten', '902-566-0344']
['CHANCES Early Learning Centre', 'Prince Edward Island', 'Charlottetown', '16 Brighton Road Charlottetown PE', 'Infant Toddler Kindergarten Schoolage', '902-892-8744']
['Daycare', 'Prince Edward Island', 'Charlot

In [11]:
src = request_page_source("http://www.godaycare.com/quebec")
prov_terr_bs = BeautifulSoup(src, 'lxml')
# Get list of city urls for the province or territory
prov_terr_city_urls = get_city_urls(prov_terr_bs)

# Loop through collected city urls
for city in prov_terr_city_urls:
    city_src = request_city_page_source(city)

    # Create Beautiful Soup Object for City Page
    city_bs = BeautifulSoup(city_src, 'lxml')

    # Get City Day Care URLS
    day_care_urls = get_city_day_care_urls(city_bs)

    # Loop through day care urls
    for day_care_url in day_care_urls:
        # Request Day Care Source
        dc_src = request_day_care_page_source(day_care_url)

        # Create Beautiful Soup Object for Day Care Page
        dc_bs = BeautifulSoup(dc_src, 'lxml')

        # Get Day Care Details
        details = get_day_care_details(dc_bs)

        if details:
            CSV_DATA.append(details)

["Ms. Lyne's Pre-K", 'Quebec', 'Anjou', '7961 Dalkeith Ave Anjou QC', 'Preschool', '514-928-2587']
['123 ABC', 'Quebec', 'Aylmer', '111 Pink Aylmer QC', '', '']
["Addy's In-Home Daycare!", 'Quebec', 'Aylmer', 'Rue Helenore Aylmer QC', 'Toddler Kindergarten Preschool Schoolage', '613 878-0410']
["Andy's Daycare", 'Quebec', 'Aylmer', 'Mulligan Aylmer QC', 'Infant Toddler Preschool', '819-684-6721']
['Arc En Ciel', 'Quebec', 'Aylmer', '278 Des Artisans Aylmer QC', 'Infant Toddler Kindergarten Preschool Schoolage', '613.875.7021']
['Aylmer Home Daycare', 'Quebec', 'Aylmer', 'right Aylmer QC', 'Toddler Kindergarten Preschool', '']
['Aylmer Home Daycare', 'Quebec', 'Aylmer', 'Gerald Dubois Aylmer QC', 'Toddler Kindergarten Preschool', '819-684-6453']
["Barb's Daycare", 'Quebec', 'Aylmer', 'Gordon Street Aylmer QC', '', '']
["Cathleen's Child Care", 'Quebec', 'Aylmer', 'rue de la terrasse eardley Aylmer QC', 'Toddler Kindergarten Preschool Schoolage', '819-684-2284']
['Chez Moi-private Daycar

['Baby Bear Daycare', 'Quebec', 'Dollard-des-ormeaux', 'spring garden Dollard-des-ormeaux QC', 'Infant Toddler Kindergarten Preschool Schoolage', '5145015330']
['BABY LUV GARDERIE', 'Quebec', 'Dollard-des-ormeaux', 'West Acres Cr. Dollard-des-Ormeaux QC', 'Infant Toddler', '514-620-8601']
["Claudia's Day Care", 'Quebec', 'Dollard-des-ormeaux', 'McKinley Dollard-des-ormeaux QC', 'Infant Toddler Preschool', '514-626-3599']
["Claudia's Daycare", 'Quebec', 'Dollard-des-ormeaux', '22 Rue McKinley Dollard-des-Ormeaux QC', 'Infant Toddler Preschool', '438-995-5402']
['Dollard-des-Ormeaux Home Daycare', 'Quebec', 'Dollard-des-ormeaux', 'Ceres Dollard-des-Ormeaux QC', 'Infant Toddler Preschool', '514-685-1935']
['Garderie A Kids World St-Jean', 'Quebec', 'Dollard-des-ormeaux', '3677 Boulevard St-Jean Dollard-des-Ormeaux QC', 'Infant Toddler Kindergarten Preschool', '514-624-6363']
['Garderie Angela', 'Quebec', 'Dollard-des-ormeaux', 'Frontenac Dollard-des-Ormeaux QC', 'Infant Toddler', '514-620

['Daycare In Kirkland West Island', 'Quebec', 'Kirkland', 'Sphinx Kirkland QC', 'Infant Toddler Preschool', '(514)6710497']
['My First Steps', 'Quebec', 'Kirkland', '208 Rue Beacon Kirkland QC', 'Toddler Preschool', '(514) 505 1924']
['West Island Home Daycare', 'Quebec', 'Kirkland', 'Rue Du Boise Kirkland QC', 'Infant Toddler Preschool', '5146939431']
['Centre Educatif A Ciel Ouvert', 'Quebec', "L'ile-bizard", "1199 Montee de l'Eglise L'Ile-Bizard QC", 'Infant Toddler Preschool', '514-624-1244']
['Little Flowers ($7.75)', 'Quebec', "L'ile-bizard", "Lavigne L'Ile-Bizard QC", 'Infant Toddler Preschool', '514-626-0741']
['AcadÃ\x83Â©mie PrÃ\x83Â©scolaire Kreative Kidz', 'Quebec', 'La Prairie', '1085 Ch. St-Jean La Prairie QC', 'Infant Toddler Preschool', '450-619-7205']
['Au Coin Montessori', 'Quebec', 'La Prairie', 'LEOPOLD-PERRON La Prairie QC', 'Infant Toddler', '514 7092669']
['GARDERIE AU COIN MONTESSORI', 'Quebec', 'La Prairie', 'LEOPOLD-PERRON J5R 0G3 La Prairie QC', 'Infant Toddl

['Academie Prescolaire Mon Depart', 'Quebec', 'Montreal', '5180 Queen-Mary suite #450 Montreal QC', 'Toddler Kindergarten Preschool Schoolage', '(514)488-3456']
['Academie Prescolaire Mont Royal', 'Quebec', 'Montreal', '4480 Cote de Liesse, suite 190 Montreal QC', 'Infant Toddler Preschool', '514 937 1662']
["Cynthia's Daycare", 'Quebec', "Notre Dame De L'ile Perrot", "Francois Rapin Notre Dame De L'Ile Perrot QC", 'Infant Toddler Preschool', '514-425-3389']
['Garderie Sur  Le Lac', 'Quebec', "Notre Dame De L'ile Perrot", "william rozon Notre Dame De L'ile Perrot QC", 'Infant Toddler', '5149415483']
['Garderie Toto Et Nini', 'Quebec', "Notre Dame De L'ile Perrot", "1038 Blv. Virginie Roy Notre Dame De L'Ile Perrot QC", 'Kindergarten', '5142994441']
["Hana's Daycare", 'Quebec', "Notre Dame De L'ile Perrot", "1282 Virginie-Roy Notre Dame De L'ile Perrot QC", 'Infant Toddler Preschool', '514-453-1624']
["Lori's Building Blocks", 'Quebec', "Notre Dame De L'ile Perrot", "boul Virginie Roy N

['Les Petits Hiboux', 'Quebec', 'Roxboro', '97 1st Ave. N. Roxboro QC', 'Toddler Preschool', '514-684-2493']
['Centre De La Petite Enfance Rayons De Soleil De Roxton Pond', 'Quebec', 'Roxton Pond', "800 rue de l'Avenir Roxton Pond QC", 'Infant Toddler Kindergarten Preschool', '450-372-7117']
['Little Sweeties (Kuya Bunso And Friends)', 'Quebec', 'Saint Prosper', '25e Avenue Saint Prosper QC', 'Infant Toddler Preschool', '4185945514']
['Academy Montessori A.M.I.Bright Hearts', 'Quebec', 'Saint-laurent', '333 Suite #323 Blvd Decarie Saint-laurent QC', 'Infant Toddler Preschool', '514-744-2220']
['GARDERIE INNOVATION', 'Quebec', 'Saint-laurent', '2311 BOUL. ALFRED NOBEL SUITE 102 Saint-Laurent QC', 'Infant Toddler Preschool', '5143346695']
['Garderie Les Jolies Frimousses', 'Quebec', 'Saint-laurent', '716 blvd Decarie Saint-Laurent QC', 'Infant Toddler Preschool', '514-748-1111']
['Mother Bear', 'Quebec', 'Saint-laurent', 'gohier Saint-Laurent QC', 'Infant Toddler', '514-800-0898']
['Tood

["Steffy's Daycare", 'Quebec', 'Vaudreuil-sur-le-lac', 'Maurice-Duplessis Vaudreuil-sur-le-Lac QC', 'Infant Toddler Preschool', '']
['Brightstars Home Daycare', 'Quebec', 'Verdun', 'Manning verdun QC', 'Infant Toddler Preschool', '438 375 9218']
['-', 'Quebec', 'Verdun', 'rue Galt Verdun QC', 'Infant Toddler Preschool', '']
['Brightstars Home Daycare', 'Quebec', 'Verdun', 'Manning verdun QC', 'Infant Toddler Preschool', '438 375 9218']
['EAdaycare', 'Quebec', 'Verdun', '1st ave Verdun QC', 'Toddler Preschool', '489397637']
['Garderie Educative Assila', 'Quebec', 'Verdun', '3901 Rue Verdun Verdun QC', 'Infant Toddler Preschool', '5145075595']
['Garderie Educative Mon Petit Monde A Verdun', 'Quebec', 'Verdun', '4348 Blvd LaSalle Verdun QC', 'Infant Toddler Preschool', '514-502-4458']
['Garderie Ohana', 'Quebec', 'Verdun', 'Lasalle Blvd. Verdun QC', 'Infant Toddler Preschool', '514-767-7301']
["Kids' Nest Daycare", 'Quebec', 'Verdun', '5614 Rue Verdun Verdun QC', 'Toddler Kindergarten Pre

In [15]:

def write_csv_data(row_data):
    """
    Write scrapped data to a csv file
    """
    csv_header = ["Name", "Prov_Terr", "City", "Address", "Age_Groups", "Telephone"]
    with open(OUTPUT_FILENAME, 'w', encoding=ENCODING) as file:
        write = csv.writer(file)
        write.writerow(csv_header)
        write.writerows(row_data)

        file.close()
        
# Output CSV Data
write_csv_data(CSV_DATA)

In [19]:
import pandas as pd
df = pd.DataFrame(CSV_DATA)
df.columns = ["Name", "Prov_Terr", "City", "Address", "Age_Groups", "Telephone"]

In [22]:
df.to_csv('data/childcare/odec.csv', encoding = 'iso-8859-1')